In [ ]:
import cPickle as pkl
from sklearn.metrics import f1_score

def read_log(folder):
    with open(os.path.join(folder, 'train.log')) as f:
        logs = {}
        for l in f:
            d = json.loads(l.strip("\n"))
            for k, v in d.items():
                if k not in logs:
                    logs[k] = []
                logs[k] += [v]
    return logs

def read_args(folder):
    with open(os.path.join(folder, 'args.json')) as f:
        return json.load(f)

def read_test(folder):
    with open(os.path.join(folder, 'test.json')) as f:
        test = json.load(f)
    with open(os.path.join(folder, 'test.pkl')) as f:
        d = pkl.load(f)
    pred, targ = d['pred'], d['targ']
    
    counts = Counter(targ.tolist())
    order = np.array([e[0] for e in counts.most_common()])
    counts = np.array([e[1] for e in counts.most_common()])
    
    cm = confusion_matrix(targ, pred, order)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    f1_macro = f1_score(targ, pred, average='macro')
    f1_micro = f1_score(targ, pred, average='micro')
    return test, cm, counts, order, f1_macro, f1_micro

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)


def plot_confusion_matrix(ax, cm, order, title='Confusion matrix', cmap=P.cm.Blues):
    names = [d.vocab['rel'].index2word[o] for o in order]
    handle = ax.imshow(cm, interpolation='nearest', cmap=matplotlib.cm.Blues)
    ax.set_title(title)
    P.colorbar(handle, ax=ax)
    tick_marks = np.arange(len(names))
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(names, rotation=90)
    ax.set_yticklabels(names)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    P.tight_layout()
    

def print_experiment(e):
    args = read_args(e)
    logs = read_log(e)
    test, cm, counts, order, f1_macro, f1_micro = read_test(e)
    args.update(test)
    args.update({'f1_macro': f1_macro, 'f1_micro': f1_micro})
    
    fig, ax = P.subplots(2, 2, figsize=(15, 15))
    
    desc = ax[0][0]
    conf = ax[0][1]
    acc = ax[1][0]
    hist = ax[1][1]
    
    desc.text(0.05, 0.95, pformat(args, indent=2), transform=desc.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)
    
    plot_histogram(hist, counts, order)
    plot_confusion_matrix(conf, cm, order)
    acc.plot(logs['train_acc'], label='train', color='b')
    acc.plot(logs['dev_acc'], label='dev', color='r')
    acc.set_ylabel('accuracy')
    acc.legend(loc='lower right')
    #     loss.plot(logs['train_loss'], label='train', color='b')
    #     loss.plot(logs['dev_loss'], label='dev', color='r')
    #     loss.set_ylabel('loss')
    #     loss.legend(loc='upper right')
    print 'processed', e

In [ ]:
P.close('all')
for e in experiments:
    print_experiment(e)